# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import holoviews as hv
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,buchanan,5.8808,-10.0467,26.78,85,98,0.86,LR,1712906411
1,1,brejo santo,-7.4933,-38.9872,24.13,93,4,0.41,BR,1712906411
2,2,flying fish cove,-10.4217,105.6791,28.82,78,40,5.66,CX,1712906412
3,3,albany,42.6001,-73.9662,16.44,81,100,3.58,US,1712906282
4,4,tarrafal,15.2833,-23.7667,20.44,83,17,5.24,CV,1712906413


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.50,
    color = "City"
    
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
max_temp_df = city_data_df.loc[(city_data_df['Max Temp'] <= 27) & (city_data_df['Max Temp'] >= 21)]
#Wind speed less than 4.5 m/s
wind_speed_df = max_temp_df.loc[(max_temp_df['Wind Speed'] <= 4.5)]
#Zero cloudiness
zero_cloudiness = wind_speed_df.loc[(wind_speed_df['Cloudiness'] == 0)]

# Drop any rows with null values
cleaned_df = zero_cloudiness.dropna()

# Display sample data
cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,103,xining,36.6167,101.7667,21.00,28,0,3.59,CN,1712906461
315,315,petatlan,17.5167,-101.2667,24.16,76,0,0.67,MX,1712906567
428,428,tazacorte,28.6290,-17.9293,23.54,20,0,4.12,ES,1712906621
517,517,quebo,11.3333,-14.9333,21.66,68,0,0.49,GW,1712906665
565,565,pimentel,-6.8367,-79.9342,24.16,69,0,3.60,PE,1712906689
571,571,aswan,24.0934,32.9070,25.61,27,0,2.06,EG,1712906691


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
103,xining,CN,36.6167,101.7667,28,
315,petatlan,MX,17.5167,-101.2667,76,
428,tazacorte,ES,28.6290,-17.9293,20,
517,quebo,GW,11.3333,-14.9333,68,
565,pimentel,PE,-6.8367,-79.9342,69,
571,aswan,EG,24.0934,32.9070,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20,  
    "radius": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
xining - nearest hotel: Qinghai hotel
petatlan - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
quebo - nearest hotel: No hotel found
pimentel - nearest hotel: Hospedaje Calle del Sol
aswan - nearest hotel: Oscar Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
103,xining,CN,36.6167,101.7667,28,Qinghai hotel
315,petatlan,MX,17.5167,-101.2667,76,No hotel found
428,tazacorte,ES,28.6290,-17.9293,20,App Leyma
517,quebo,GW,11.3333,-14.9333,68,No hotel found
565,pimentel,PE,-6.8367,-79.9342,69,Hospedaje Calle del Sol
571,aswan,EG,24.0934,32.9070,27,Oscar Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "ESRI",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.50,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)